In [ ]:
!pip install earthpy

In [2]:
from glob import glob

import earthpy as et
import earthpy.spatial as es
import earthpy.plot as ep

import rasterio as rio

import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
import pandas as pd
from matplotlib.colors import ListedColormap

import plotly.graph_objects as go

import json
import os
import cv2

np.seterr(divide='ignore', invalid='ignore')

In [3]:
samples1 = np.load('../input/deforestation/test/private/1613865782655850287303.npy')
# samples1 = np.moveaxis(samples1, [0,1,2], [1,2,0])

In [4]:
samples1.shape

In [5]:
pixels = samples1[:, :, [1,2,3]]
pixels.shape

In [6]:
samples1 = np.moveaxis(samples1, [0,1,2], [1,2,0])

In [7]:
print(f'maax : {np.max(pixels[:, :,0])}, min : {np.min(pixels[:, :,0])}')

In [8]:
print(samples1.shape)
ep.plot_bands(samples1, 
              cmap = 'gist_earth', 
              figsize = (20, 12), 
              cols = 6, 
              cbar = False)
plt.show()

In [9]:
print(pixels.shape)

In [10]:
ep.plot_bands(np.moveaxis(pixels, [0,1,2], [1,2,0]), 
              cmap = 'gist_earth', 
              figsize = (20, 12), 
              cols = 6, 
              cbar = False)
plt.show()

In [11]:
# RGB Composite Image

rgb = ep.plot_rgb(samples1, 
                  rgb=(3,2,1), 
                  figsize=(10, 16))
plt.show()

In [12]:
# RGB Composite Image with Strech

ep.plot_rgb(samples1,
            rgb=(3, 2, 1),
            stretch=True,
            str_clip=0.2,
            figsize=(10, 16))
plt.show()

In [13]:
samples3 = np.load('../input/deforestation/train/public/161386738320912167535.npy')
samples3 = np.moveaxis(samples3, [0,1,2], [1,2,0])
ep.plot_rgb(samples3,
            rgb=(3, 2, 1),
            stretch=True,
            str_clip=0.2,
            figsize=(10, 16))
plt.show()

In [14]:
ep.plot_bands(samples3, 
              cmap = 'gist_earth', 
              figsize = (20, 12), 
              cols = 6, 
              cbar = False)
plt.show()

In [15]:
samples4 = np.load('../input/deforestation/train/public/161386738320912117707.npy')
samples4 = np.moveaxis(samples4, [0,1,2], [1,2,0])
ep.plot_rgb(samples4,
            rgb=(3, 2, 1),
            stretch=True,
            str_clip=0.2,
            figsize=(10, 16))
plt.show()

In [16]:
ep.plot_bands(samples4, 
              cmap = 'gist_earth', 
              figsize = (20, 12), 
              cols = 6, 
              cbar = False)
plt.show()

In [17]:
samples5 = samples3-samples4
# ep.plot_rgb(samples5,
#             rgb=(3, 2, 1),
#             stretch=True,
#             str_clip=0.2,
#             figsize=(10, 16))
# rsamples5 = np.moveaxis(samples5, [0,1,2], [2,0,1])
# plt.imshow(rsamples5[:, :,[3,2,1]])
# rgb = ep.plot_rgb(samples5, 
#                   rgb=(3,2,1), 
#                   figsize=(10, 16))
ep.plot_rgb(samples5,
            rgb=(3, 2, 1),
            stretch=True,
            str_clip=0.2,
            figsize=(10, 16))
plt.show()

In [18]:
samples5 = samples4-samples3
ep.plot_bands(samples5, 
              cmap = 'gist_earth', 
              figsize = (20, 12), 
              cols = 6, 
              cbar = False)
plt.show()

In [19]:
mask1 = np.load('../input/deforestation/train/public/1613867395998297546666.npy')
mask1.shape
# samples2 = np.moveaxis(samples2, [0,1,2], [1,2,0])
# ep.plot_rgb(samples2,
#             rgb=(3, 2, 1),
#             stretch=True,
#             str_clip=0.2,
#             figsize=(10, 16))
# plt.show()
# plt.imshow(mask1, cmap='jet')
# plt.colorbar()
# plt.axis('off')
# plt.show()

In [20]:
print(np.max(mask1), np.min(mask1))

In [21]:
with open("../input/deforestation/train.json",'r') as pf:
    train_meta = json.load(pf)

In [22]:
len(train_meta)

In [23]:
with open("../input/deforestation/test.json",'r') as pf:
    test_meta = json.load(pf)

In [24]:
len(test_meta)

In [44]:
root = '../input/deforestation/train/public/'
m_max  = []
f_imgs = []
s_imgs = []
masks = []
for idx in train_meta:
    sample = train_meta[idx]
    f_paths = sample['files']
    f_img_path = f_paths['satellite_img_first']
    s_img_path = f_paths['satellite_img_second']
    mask_path = f_paths['mask']
    f_img0 = np.load(os.path.join(root, f_img_path), mmap_mode='r')
    f_img = f_img0[:, :, [3,2,1]]
    s_img = np.load(os.path.join(root, s_img_path), mmap_mode='r')[:, :, [3,2,1]]
    mask_img = np.load(os.path.join(root,  mask_path))
    
    f_img = cv2.normalize(f_img, None, 0, 255, cv2.NORM_MINMAX, dtype=cv2.CV_8U)
    s_img = cv2.normalize(s_img, None, 0, 255, cv2.NORM_MINMAX, dtype=cv2.CV_8U)
    f_img = cv2.resize(f_img, (256, 256), interpolation = cv2.INTER_NEAREST )
    s_img = cv2.resize(s_img, (256, 256), interpolation = cv2.INTER_NEAREST )
    mask_img = cv2.resize(mask_img, (256, 256), interpolation = cv2.INTER_NEAREST )
    m_max.append(np.max(mask_img))
    f_imgs.append(f_img)
    s_imgs.append(s_img)
    masks.append(mask_img)
#     print(mask_img.shape, f_img0.shape, f_img.shape)
#     break

f_np, s_np, m_np = np.array(f_imgs), np.array(s_imgs), np.array(masks)

print(f_np.shape, s_np.shape, m_np.shape)

In [45]:
print(np.max(m_np))

In [46]:
thn = m_np > 1
print(np.sum(thn))

In [47]:
print(np.unique(m_np).shape)

In [29]:
ep.plot_bands(np.moveaxis(f_np[0],[0,1,2],[2,1,0]), 
              cmap = 'gist_earth', 
              figsize = (20, 12), 
              cols = 6, 
              cbar = False)
plt.show()

In [30]:
f_np[0].shape

In [31]:
import matplotlib.pyplot as plt
import cv2


In [48]:
# rescale_image = cv2.normalize(f_np[0], None, 0, 255, cv2.NORM_MINMAX, dtype=cv2.CV_8U)
plt.imshow(f_np[0])

In [49]:
# rescale_image = cv2.normalize(s_np[0][..., [0,1,2]], None, 0, 255, cv2.NORM_MINMAX, dtype=cv2.CV_8U)
plt.imshow(s_np[0])

In [50]:
npz_file = './image_triples.npz'
np.savez_compressed(npz_file, first_image=f_np, second_image=s_np, mask=m_np)

In [ ]:
np.load("./image_triples.npz")

In [ ]:
import h5py

In [ ]:
hf = h5py.File('./data.h5', 'w')
hf.create_dataset('first_img', data=f_np)
hf.create_dataset('second_img', data=s_np)
hf.create_dataset('mask', data=m_np)
hf.close()

In [ ]:
with open("./sample.txt", 'w') as pf:
    pf.write(" Hello  ")

In [ ]:
ff_np = np.expand_dims(f_np, 0)
sf_np = np.expand_dims(s_np, 0)
c_np = np.vstack((ff_np, sf_np))
n_np = np.moveaxis(c_np, [0,1,2,3], [3,0,1,2])
print(n_np.shape)

model

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import cv2
from scipy import io
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt 

In [ ]:
# train_X1 = tf.data.Dataset.from_tensor_slices(np.expand_dims(f_np, -1))
train_X = tf.data.Dataset.from_tensor_slices(n_np)
mask_y = tf.data.Dataset.from_tensor_slices(np.expand_dims(m_np, -1))

In [ ]:
train = tf.data.Dataset.zip((train_X, mask_y))
train.element_spec

In [ ]:
def brightness(img1, mask):
    img1 = tf.image.adjust_brightness(img1, 0.1)
#     img2 = tf.image.adjust_brightness(img2, 0.1)
    return img1, mask

def gamma(img1, mask):
    img1 = tf.image.adjust_gamma(img1, 0.1)
#     img2 = tf.image.adjust_gamma(img2, 0.1) 
    return img1, mask

def hue(img1, mask):
    img1 = tf.image.adjust_hue(img1, -0.1)
#     img2 = tf.image.adjust_hue(img2, -0.1)
    return img1, mask

# def crop(img, mask):
#      img = tf.image.central_crop(img, 0.7)
#      img = tf.image.resize(img, (128,128))
#      mask = tf.image.central_crop(mask, 0.7)
#      mask = tf.image.resize(mask, (128,128))
#      mask = tf.cast(mask, tf.uint8)
#      return img, mask

def flip_hori(img1, mask):
    img1 = tf.image.flip_left_right(img1)
#     img2 = tf.image.flip_left_right(img2)
    mask = tf.image.flip_left_right(mask)
    return img1, mask

def flip_vert(img1, mask):
    img1 = tf.image.flip_up_down(img1)
#     img2 = tf.image.flip_up_down(img2)
    mask = tf.image.flip_up_down(mask)
    return img1, mask

def rotate(img1, mask):
    img1 = tf.image.rot90(img1)
#     img2 = tf.image.rot90(img2)
    mask = tf.image.rot90(mask)
    return img1, mask

In [ ]:
# perform augmentation on train data only
a = train.map(brightness)
b = train.map(gamma)
e = train.map(flip_hori)
f = train.map(flip_vert)
g = train.map(rotate)

train = train.concatenate(a)
train = train.concatenate(b)
train = train.concatenate(e)
train = train.concatenate(f)
train = train.concatenate(g) 

In [ ]:
train.element_spec

In [ ]:
BATCH = 64
AT = tf.data.AUTOTUNE
BUFFER = 1000
STEPS_PER_EPOCH = 800//BATCH
VALIDATION_STEPS = 200//BATCH
train = train.cache().shuffle(BUFFER).batch(BATCH).repeat()
train = train.prefetch(buffer_size=AT)

In [ ]:
from tensorflow.keras.initializers import RandomNormal, HeUniform
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, Flatten, Dense, Conv2DTranspose, LeakyReLU, Activation, Dropout, BatchNormalization, LeakyReLU, GlobalMaxPool2D, Concatenate, ReLU, AveragePooling2D
from tensorflow.keras import losses

In [ ]:

def define_encoder_block(layer_in, n_filters, batchnorm=True):
    init = HeUniform()
    g = Conv2D(n_filters, (4, 4), strides=(2, 2), padding='same', kernel_initializer=init)(layer_in)
    if batchnorm:
        g =  BatchNormalization()(g, training=True)
    g = LeakyReLU(alpha=0.2)(g)
    return g

def define_decoder_block(layer_in, skip_in, n_filters, dropout=True):
    init = RandomNormal(stddev=0.02)
    g = Conv2DTranspose(n_filters, (4, 4), strides=(2,2), padding='same', kernel_initializer=init)(layer_in)
    g = BatchNormalization()(g, training=True)
    if dropout:
        g = Dropout(0.4)(g, training=True)
    g = Concatenate()([g, skip_in])
    g = ReLU()(g)
    return g

def define_generator(latent_size, image_shape=(128, 128, 2)):
    init = RandomNormal(stddev=0.02)
    input_image = Input(shape=image_shape)
#     style_image = Input(shape=image_shape)
    # stack content and style images
#     stacked_layer = Concatenate()([content_image, style_image])
    #encoder model
    e1 = define_encoder_block(input_image, 64, batchnorm=False)
    e2 = define_encoder_block(e1, 128)
    e3 = define_encoder_block(e2, 256)
    e4 = define_encoder_block(e3, 512)
    e5 = define_encoder_block(e4, 512)
    e6 = define_encoder_block(e5, 512)
    #e7 = define_encoder_block(e6, 512)
    # bottleneck layer
    b = Conv2D(latent_size, (4, 4), strides=(2, 2), padding='same', kernel_initializer=init)(e6)
    b = ReLU()(b)
    #decoder model
    #d1 = define_decoder_block(b, e7, 512)
    d2 = define_decoder_block(b, e6, 512)
    d3 = define_decoder_block(d2, e5, 512)
    d4 = define_decoder_block(d3, e4, 512, dropout=False)
    d5 = define_decoder_block(d4, e3, 256, dropout=False)
    d6 = define_decoder_block(d5, e2, 128, dropout=False)
    d7 = define_decoder_block(d6, e1, 64, dropout=False)
    #output layer
    g = Conv2DTranspose(1, (4, 4), strides=(2, 2), padding='same', kernel_initializer=init)(d7)
    out_image = Activation('sigmoid')(g)
    model = Model(inputs=input_image, outputs=out_image, name='generator')
    return model

In [ ]:
model = define_generator(32, (256, 256, 1))
# tf.keras.utils.plot_model(model, show_shapes=True)

In [ ]:
samples = np.ones((10, 256, 256, 2))
res = model(samples)

In [ ]:
model.compile(loss= keras.losses.BinaryCrossentropy(from_logits=True),
             optimizer=keras.optimizers.Adam(lr=0.001),
             metrics=['accuracy']) 

hist = model.fit(x = train,
            steps_per_epoch=STEPS_PER_EPOCH,
            epochs=50)